In [151]:
!pip install pyreadr

In [152]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadr
import numpy as np

# Data cleaning

# Collect data

Dataset documentation : "https://cas.uqam.ca/pub/web/CASdatasets-manual.pdf"

Here, we concentrate on 3 different datasets on Motor Insurers : 
- `freMPL` (*French Motor Personal Line datasets*): This collection of ten datasets comes from a private motor French insurer. Each dataset includes risk features, claim amount and claim history of around 30,000 policies for year 2004. We decided to concacenate the differents freMPL datasets to focus on statistics.
- `beMTPL16` (*Belgian motor third-part liability dataset*): The dataset beMTPL was collected by an unknown Belgium insurer. It consists of 70 791 claims for private motor insurance.
- `euMTPL` (*European motor TPL insurance*): The euMTPL compiles three years of experience from a European MTPL (Motor Third Party Liability) portfolio, including frequency and severity values for different types of losses. The data was collected during the first decade of the 21st century.

In [153]:
# Change the paths to the location of the data on your machine
be_path = "/home/onyxia/work/Federated_Learning_Milliman/data_comparaison/beMTPL97.rda"
eu_path = "/home/onyxia/work/Federated_Learning_Milliman/data_comparaison/euMTPL.rda"
fre_path = "/home/onyxia/work/Federated_Learning_Milliman/data_comparaison/freMPL.csv"
file_paths = [be_path, eu_path]

In [154]:
def load_rda(file_paths):
  dataframes = {}
  for file_path in file_paths:
      result = pyreadr.read_r(file_path)
      for key in result.keys():
          dataframes[key] = result[key]
  return dataframes

In [155]:
df_fre=pd.read_csv(fre_path)

In [156]:
df_0 = load_rda(file_paths)
dataframes = {
    "French": df_fre,
    "Belgium": df_0["beMTPL97"],
    "European": df_0["euMTPL"]
}

In [157]:
print("French columns:", dataframes["French"].columns)
print("Belgium columns:", dataframes["Belgium"].columns)
print("European columns:", dataframes["European"].columns)


French columns: Index(['Exposure', 'LicAge', 'RecordBeg', 'Gender', 'MariStat', 'SocioCateg',
       'VehUsage', 'DrivAge', 'HasKmLimit', 'BonusMalus', 'ClaimAmount',
       'ClaimInd'],
      dtype='object')
Belgium columns: Index(['id', 'expo', 'claim', 'nclaims', 'amount', 'average', 'coverage',
       'ageph', 'sex', 'bm', 'power', 'agec', 'fuel', 'use', 'fleet',
       'postcode', 'long', 'lat'],
      dtype='object')
European columns: Index(['policy_id', 'group', 'fuel_type', 'year', 'vehicle_category',
       'vehicle_use', 'province', 'horsepower', 'gender', 'age', 'exposure',
       'cost_nc', 'num_nc', 'cost_cg', 'num_cg', 'cost_fcg', 'num_fcg',
       'cost_cd', 'num_cd'],
      dtype='object')


In [158]:
print("French columns:", np.shape(dataframes["French"]))
print("be columns:", np.shape(dataframes["Belgium"]))
print("eu columns:", np.shape(dataframes["European"]))


French columns: (243065, 12)
be columns: (163212, 18)
eu columns: (2373197, 19)


In [159]:
dataframes["European"]["cost_claim"] = dataframes["European"]["cost_nc"] + dataframes["European"]["cost_cg"] + dataframes["European"]["cost_fcg"] + dataframes["European"]["cost_cd"] 


In [160]:
dataframes["European"]["cost_claim"].describe()

count    2.373197e+06
mean     3.058943e+02
std      4.005364e+03
min     -1.034309e+04
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.452709e+06
Name: cost_claim, dtype: float64

# Choice of the variables to compare & Data Cleaning

Since the datasets are not identical, we need to select a common variable that will serve as the basis for comparison.
The target target variables are : 
- `Exposure` (*French :*`Exposure`, *Belgium :*`expo`, *European :*`exposure`): a numeric time of exposure, in years. -- **integer**
- `DriverAge` (*French :*`DrivAge`, *Belgium :*`ageph`, *European :*`age`): age of the driver -- **integer**
- `Gender` (*French :*`Gender`, *Belgium :*`sex`, *European :*`gender`): gender of the driver -- **{Male: 0,Female: 1}**
- `VehiculeUsage` (*French :*`VehUsage`, *Belgium :*`use`, *European :*`vehicle_use`): age of the driver -- **integer**
- `ClaimAmout` (*French :*`ClaimAmount`, *Belgium :*``, *European :*`cost_...`(differnt variables)):Total claim amount of the guarantee -- **real number**. For the european dataset, We will sum all the different `cost`variables to agregate all the sinisters.

Since the datasets have different variables, it is challenging to compare them accurately. Moreover, the French database had to be concatenated, and some variables had to be removed, for example.

In [161]:
french_data = dataframes["French"].rename(columns={
    'Exposure': 'Exposure',
    'DrivAge': 'DriverAge',
    'Gender': 'Gender',
    'VehUsage': 'VehiculeUsage',
    'ClaimAmount': 'ClaimAmount'
})[['Exposure', 'DriverAge', 'Gender', 'VehiculeUsage', 'ClaimAmount']]

belgium_data = dataframes["Belgium"].rename(columns={
    'expo': 'Exposure',
    'ageph': 'DriverAge',
    'sex': 'Gender',
    'use': 'VehiculeUsage',
    'amount': 'ClaimAmount'
})[['Exposure', 'DriverAge', 'Gender', 'VehiculeUsage', 'ClaimAmount']]

european_data = dataframes["European"].rename(columns={
    'exposure': 'Exposure',
    'age': 'DriverAge',
    'gender': 'Gender',
    'vehicle_use': 'VehiculeUsage',
    'cost_claim': 'ClaimAmount'
})[['Exposure', 'DriverAge', 'Gender', 'VehiculeUsage', 'ClaimAmount']]


### Sample the European data to approximize the size of the French and Belgium data

In [162]:
reduction_factor = 10
sample_size = len(european_data) // reduction_factor
european_data = european_data.sample(sample_size)

french_size = len(french_data)
belgium_size = len(belgium_data)
european_size = len(european_data)

total_size = french_size + belgium_size + european_size

french_proportion = 100 * (french_size / total_size)
belgium_proportion = 100 * (belgium_size / total_size)
european_proportion = 100 * (european_size / total_size)

print("French Proportion :", round(french_proportion, 2), "%")
print("Belgium Proportion :", round(belgium_proportion, 2), "%")
print("European Proportion :", round(european_proportion, 2), "%")


French Proportion : 37.77 %
Belgium Proportion : 25.36 %
European Proportion : 36.87 %


### Rename variables values

In [163]:
french_data['Sinistre'] = (french_data['ClaimAmount'] > 0).astype(int)
belgium_data['Sinistre'] = (belgium_data['ClaimAmount'] > 0).astype(int)
european_data['Sinistre'] = (european_data['ClaimAmount'] > 0).astype(int)

french_data['Gender'] = french_data['Gender'].replace({'Female': 1, 'Male': 0})
belgium_data['Gender'] = belgium_data['Gender'].replace({'female': 1, 'male': 0})
european_data['Gender'] = european_data['Gender'].replace({'F': 1, 'M': 0}).astype(int)

french_data['VehiculeUsage'] = french_data['VehiculeUsage'].map({
    'Private' : 1,
    'Private+trip to office': 1,
    'Professional' : 0,
    'Professional run': 0})

belgium_data['VehiculeUsage'] = belgium_data['VehiculeUsage'].map({
    'private' : 1,
    'work' : 0})

# Regroupin 0,4,5,26 values in european data
european_data['VehiculeUsage'] = european_data['VehiculeUsage'].astype(int).replace({4: 0, 5: 0, 26: 0})


# french_data['Dataset'] = 'french'
# belgium_data['Dataset'] = 'belgium'
# european_data['Dataset'] = 'european'



# df_combined=pd.concat([french_data, belgium_data, european_data])


/tmp/ipykernel_93611/2461441814.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  french_data['Gender'] = french_data['Gender'].replace({'Female': 1, 'Male': 0})
/tmp/ipykernel_93611/2461441814.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  belgium_data['Gender'] = belgium_data['Gender'].replace({'female': 1, 'male': 0})
/tmp/ipykernel_93611/2461441814.py:6: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that

In [164]:
french_data.to_csv('/home/onyxia/work/Federated_Learning_Milliman/data_clean/freMPL_common', index = False)
belgium_data.to_csv('/home/onyxia/work/Federated_Learning_Milliman/data_clean/beMTPL_common', index = False)
european_data.to_csv('/home/onyxia/work/Federated_Learning_Milliman/data_clean/euMTPL_common', index = False)

In [172]:
test = pd.read_csv('/home/onyxia/work/Federated_Learning_Milliman/data_clean/beMTPL_common')

In [173]:
test

,Exposure,DriverAge,Gender,VehiculeUsage,ClaimAmount,Sinistre
0,1.000000,50,0,1,1618.001036,1
1,1.000000,64,1,1,0.000000,0
2,1.000000,60,0,1,0.000000,0
3,1.000000,77,0,1,0.000000,0
4,0.046575,28,1,1,155.974606,1
...,...,...,...,...,...,...
163207,1.000000,37,0,0,0.000000,0
163208,1.000000,44,0,1,0.000000,0
163209,1.000000,50,0,1,0.000000,0
163210,1.000000,43,0,1,0.000000,0


In [174]:
test.describe()

,Exposure,DriverAge,Gender,VehiculeUsage,ClaimAmount,Sinistre
count,163212.000000,163212.000000,163212.000000,163212.000000,163212.000000,163212.000000
mean,0.889744,47.000950,0.264490,0.951731,162.150883,0.111977
std,0.244202,14.831561,0.441063,0.214334,1374.575776,0.315339
min,0.002740,18.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,35.000000,0.000000,1.000000,0.000000,0.000000
50%,1.000000,46.000000,0.000000,1.000000,0.000000,0.000000
75%,1.000000,58.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,95.000000,1.000000,1.000000,140032.424473,1.000000
